<a href="https://colab.research.google.com/github/marco-siino/PAN-CRYPTO-2023/blob/main/subtask1/training/PAN_CRYPTO_TRAINING_XLNET_AUG_JA_SUBTASK_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and import the needed modules

In [1]:
!pip install simpletransformers
!pip install tensorflow-addons
!pip install -U deep-translator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.5/206.5 kB 14

In [2]:
import numpy as np
import tensorflow_addons as tfa
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.model_selection import KFold, StratifiedKFold
from google.colab import files
from deep_translator import GoogleTranslator
from urllib import request

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [3]:
# Import class Vectorizer
module_url = f"https://raw.githubusercontent.com/marco-siino/PAN-CRYPTO-2023/main/subtask1/training/crypto_vectorzer.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))
from crypto_vectorzer import Vectorizer

# Import class Dataset
module_url = f"https://raw.githubusercontent.com/marco-siino/PAN-CRYPTO-2023/main/subtask1/training/crypto_dataset.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))
from crypto_dataset import Dataset

#Import class Simulator
module_url = f"https://raw.githubusercontent.com/marco-siino/PAN-CRYPTO-2023/main/subtask1/training/simulator_kfold.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))
from simulator_kfold import Simulator

Fetching https://raw.githubusercontent.com/marco-siino/PAN-CRYPTO-2023/main/subtask1/training/crypto_vectorzer.py
Fetching https://raw.githubusercontent.com/marco-siino/PAN-CRYPTO-2023/main/subtask1/training/crypto_dataset.py
Fetching https://raw.githubusercontent.com/marco-siino/PAN-CRYPTO-2023/main/subtask1/training/simulator_kfold.py
Fetching https://raw.githubusercontent.com/sagacemente/PAN_2023_crypto/main/crypto_vectorzer.py


# Load the dataset 

In [4]:
url = 'https://github.com/marco-siino/PAN-CRYPTO-2023/raw/main/subtask1/training/pan23-profiling-cryptocurrency-influencers-augmented-ja.zip'
ds = Dataset(url, n_subtask = 1, language='ja')
ds.build_ds(1)
xtr, xte = ds.get_train_test_df()

# CHECK SAMPLES
N = 2
for i in ds.test_set.take(N):
  print(i)

233764/233764 [==============================] - 0s 0us/step
id_label_dict {'0037a672f0ed64b3231bac64853a278d': 'nano', '03eaa72711143b521c073d9ac5745923': 'nano', '0409fe210a0edfe258d21e3404e1ce05': 'micro', '05ca545f2f700d0d5c916657251d010b': 'mega', '062492818c984febba843b650a4a602e': 'nano', '0d3700fa5c7c3fce6fd1e1ffd5282f50': 'macro', '0e1d2c43b93e8e80dc8eb6b29d48b2c1': 'no influencer', '0ed9637249db91cb2c256ec156ce1977': 'no influencer', '0f0942696ae8bcadf0db494cce7333e0': 'nano', '11727505346c5f966891f74094042073': 'nano', '1316cba6e1442a126102d5e2ab0813ec': 'macro', '13e873428de3ccfd3873b3d7258ad411': 'macro', '179aa6d5d94999ae0bf77d13bcc5fa31': 'mega', '1bc1201be24ca9d0c0afd94ca0116609': 'nano', '1bc3271b139a2c06af59e579890796f7': 'nano', '1cbf16d93cc6a672eaed9c16ac07f114': 'mega', '1fe087134fe681f8509f8d9ced432621': 'macro', '21658f2f073f0e041b79bf0f7632a0eb': 'mega', '238a39478f2f7b782dfdbeee63df503f': 'no influencer', '28ab93d1cfa1452bee4da4a93746b956': 'micro', '2a45947271

# Run Electra

In [5]:
simulator = Simulator("xlnet", num_fold =5, nr_epochs=30, ds=ds, vectorize_layer=['NO'], num_labels=5)
simulator.run()

Transformer Setup completed

Setup for Roberta completed.


Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.bias', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a


EPOCH NUMBER:  0

NOW TRAIN THE MODEL.


  0%|          | 0/160 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

...

Running Epoch 0 of 1:   0%|          | 0/10 [00:00<?, ?it/s]


NOW EVALUATE THE TEST DF.


  0%|          | 0/160 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/160 [00:00<?, ?it/s]

{'mcc': 0.9768964572388101, 'acc': 0.9810264052887003, 'eval_loss': 0.030466026067733763}
Macro F1 on test set is: 0.9810264052887003 


Accuracy Over epochs [0.1716659275683666, 0.267068040586183, 0.37796826849652243, 0.6181069203004322, 0.5703817759367771, 0.7449611617696724, 0.7437765863900165, 0.8487387239626045, 0.815494269270013, 0.8994596028713893, 0.8867924061452639, 0.9123165424431102, 0.9250256519397505, 0.9621223007453148, 0.9378662215972075, 0.956610820873116, 0.9563858855211945, 0.9811430146914019, 0.9874984737484738, 0.9752246253692733, 0.9937484737484737, 0.9811430146914019, 0.9379683204239132, 0.9937484737484737, 0.9937484737484737, 0.9937484737484737, 1.0, 1.0, 0.9937484737484737, 0.9810264052887003]


 Over all runs maximum accuracies are: [0.1716659275683666, 0.267068040586183, 0.37796826849652243, 0.6181069203004322, 0.6181069203004322, 0.7449611617696724, 0.7449611617696724, 0.8487387239626045, 0.8487387239626045, 0.8994596028713893, 0.8994596028713893, 0.912316542

In [6]:
# Print a sample from the dataset.
for i in ds.test_set.take(19):
  continue

print(i)
print(i[0][0])

(<tf.Tensor: shape=(1,), dtype=string, numpy=
array([b'RT @HiNFTGiveaway: You\xe2\x80\x99ll receive 86,000,000\n\n86,000,000 in $Shib  to the first 3000 people who follows and retweets\n\n Drop your $ETH walle\xe2\x80\xa6RT @HiNFTGiveaway: Receive 86,000,000\n\n$86,000,000 in $Shib for the first 3000 followers and retweets\n\n Drop the $ETH wall... RT @TotemFi: \xf0\x9f\x92\xa0 Announcing the $SOL Wolf Pool \xf0\x9f\x92\xa0\n\n\xf0\x9f\x92\xab To celebrate our new asset pool, we are giving away:\n\n\xf0\x9f\x9a\x80 3 $SOL \xf0\x9f\x9a\x80\n\xf0\x9f\x9a\x80 $1,000 $TOTM \xf0\x9f\x9a\x80\n\nRULES:\xe2\x80\xa6RT @TotemFi: \xf0\x9f\x92\xa0 $SOL Wolf Pool Announcement \xf0\x9f\x92\xa0\n\n\xf0\x9f\x92\xab To celebrate our new asset pool, we are giving away:\n\n\xf0\x9f\x9a\x80 3 $SOL \xf0\x9f\x9a\x80\n\xf0\x9f\x9a\x80 $1,000 $TOTM \xf0\x9f\x9a\x80\n\nrule:\xe2\x80\xa6 RT @TheNFTlucky: $500 in $ETH  (5 WINNERS)\xf0\x9f\x8e\x81\xf0\x9f\x8f\x86\n\nRT + Follow me &amp; "done"\xf0\x9f\x94\x94RT @

In [7]:
# CORRECT MAPPING LABEL-PREDICTIONS ARRAY

# ["macro" "mega" "micro" "nano" "no influencer"]

#Original order provided in random-baseline
#LABELS=['nano', 'micro', 'macro', 'mega', 'no influencer']

LABELS=['macro', 'mega', 'micro', 'nano', 'no influencer']

In [8]:
# Load the trained model.
model = ClassificationModel("xlnet", "outputs")

In [9]:
print(i[0].numpy()[0])
print(model.predict([str(i[0].numpy()[0])]))
prediction = model.predict([str(i[0].numpy()[0])])[1]
print(prediction)
print(LABELS[np.array(prediction).argmax()])

new_str='The first #Crypto Project with #MoveToEarn Auto-staking &amp;amp;amp; Auto-compounding with $GMT &amp;amp;amp; $BNB Dividends Twitter- RunEarn\u2026'

print(model.predict([new_str]))
prediction = model.predict([new_str])[1]
print(prediction)
print(LABELS[np.array(prediction).argmax()])

b'RT @HiNFTGiveaway: You\xe2\x80\x99ll receive 86,000,000\n\n86,000,000 in $Shib  to the first 3000 people who follows and retweets\n\n Drop your $ETH walle\xe2\x80\xa6RT @HiNFTGiveaway: Receive 86,000,000\n\n$86,000,000 in $Shib for the first 3000 followers and retweets\n\n Drop the $ETH wall... RT @TotemFi: \xf0\x9f\x92\xa0 Announcing the $SOL Wolf Pool \xf0\x9f\x92\xa0\n\n\xf0\x9f\x92\xab To celebrate our new asset pool, we are giving away:\n\n\xf0\x9f\x9a\x80 3 $SOL \xf0\x9f\x9a\x80\n\xf0\x9f\x9a\x80 $1,000 $TOTM \xf0\x9f\x9a\x80\n\nRULES:\xe2\x80\xa6RT @TotemFi: \xf0\x9f\x92\xa0 $SOL Wolf Pool Announcement \xf0\x9f\x92\xa0\n\n\xf0\x9f\x92\xab To celebrate our new asset pool, we are giving away:\n\n\xf0\x9f\x9a\x80 3 $SOL \xf0\x9f\x9a\x80\n\xf0\x9f\x9a\x80 $1,000 $TOTM \xf0\x9f\x9a\x80\n\nrule:\xe2\x80\xa6 RT @TheNFTlucky: $500 in $ETH  (5 WINNERS)\xf0\x9f\x8e\x81\xf0\x9f\x8f\x86\n\nRT + Follow me &amp; "done"\xf0\x9f\x94\x94RT @TheNFTlucky: $500 in $ETH (5 Winners)\xf0\x9f\x8e\x81

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

([3], array([[-2.8671875 , -1.86523438, -2.48242188,  7.8671875 , -0.24633789]]))


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

([4], array([[-1.70410156, -1.57617188, -2.76171875, -0.70947266,  7.5703125 ]]))


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[[-1.70410156 -1.57617188 -2.76171875 -0.70947266  7.5703125 ]]
no influencer


In [10]:
!zip -r outputs_xlnet_ja.zip outputs

  adding: outputs/ (stored 0%)
  adding: outputs/eval_results.txt (deflated 13%)
  adding: outputs/training_args.bin (deflated 49%)
  adding: outputs/tokenizer_config.json (deflated 50%)
  adding: outputs/config.json (deflated 55%)
  adding: outputs/pytorch_model.bin (deflated 7%)
  adding: outputs/model_args.json (deflated 62%)
  adding: outputs/spiece.model (deflated 49%)
  adding: outputs/special_tokens_map.json (deflated 52%)
  adding: outputs/tokenizer.json (deflated 75%)
  adding: outputs/checkpoint-10-epoch-1/ (stored 0%)
  adding: outputs/checkpoint-10-epoch-1/training_args.bin (deflated 49%)
  adding: outputs/checkpoint-10-epoch-1/tokenizer_config.json (deflated 50%)
  adding: outputs/checkpoint-10-epoch-1/config.json (deflated 55%)
  adding: outputs/checkpoint-10-epoch-1/pytorch_model.bin (deflated 7%)
  adding: outputs/checkpoint-10-epoch-1/model_args.json (deflated 62%)
  adding: outputs/checkpoint-10-epoch-1/optimizer.pt (deflated 25%)
  adding: outputs/checkpoint-10-epoch

In [ ]:
files.download('outputs.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>